From https://pub.towardsai.net/text-classification-with-simple-transformers-a29d13358135

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from torch.utils.tensorboard import SummaryWriter

In [3]:
os.environ['TOKENIZERS_PARALLELISM'] =  'True'
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.drop(['id', 'keyword', 'location'], axis=1, inplace=True)
test.drop(['id', 'keyword', 'location'], axis=1, inplace=True)
train.columns = ['text', 'labels']
train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train['labels'], random_state=42)

In [6]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=True, manual_seed=42, best_model_dir='./outputs_bert/', evaluate_during_training=True, wandb_project='disaster_tweets_bert')
model = ClassificationModel(model_type='bert', model_name='bert-base-cased', use_cuda=False, num_labels=2, args=model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
model.train_model(train_df, eval_df=valid_df)

  0%|          | 0/6090 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

wandb: Currently logged in as: stephenskelley. Use `wandb login --relogin` to force relogin


Running Epoch 0 of 3:   0%|          | 0/762 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

...

  0%|          | 0/1523 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/762 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

(2286,
 defaultdict(list,
             {'global_step': [762, 1524, 2000, 2286],
              'train_loss': [0.05436320975422859,
               0.3758724629878998,
               0.006658866070210934,
               0.002790168160572648],
              'mcc': [0.6660572499462316,
               0.6526520561985643,
               0.6347188413935051,
               0.6560960490887454],
              'tp': [505, 522, 521, 518],
              'tn': [770, 742, 729, 749],
              'fp': [99, 127, 140, 120],
              'fn': [149, 132, 133, 136],
              'auroc': [0.8964362003497992,
               0.8930068305866704,
               0.8888718798717637,
               0.8876947385831371],
              'auprc': [0.893688129644985,
               0.8959051336270653,
               0.8894055472116428,
               0.8908135333066198],
              'eval_loss': [0.42692956626805334,
               0.47428044785028667,
               0.7316320378720975,
               0.714274539

In [8]:
result, model_outputs, wrong_preds = model.eval_model(valid_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(valid_df['labels'], predictions), 'accuracy', accuracy_score(valid_df['labels'], predictions))

  0%|          | 0/1523 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/191 [00:00<?, ?it/s]

Training loss,▆▇▄▆▂█▆▄▅▆▄▂▃▂▁▄█▃▇▁▁▆▃▅▁▅▁▇▄▁█▁▁▁▆▁▁▁▅▄
auprc,▆█▁▃
auroc,█▅▂▁
eval_loss,▁▂██
fn,█▁▁▃
fp,▁▆█▅
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
mcc,█▅▁▆
tn,█▃▁▄
tp,▁██▆


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


f1 score: 0.8018575851393189 accuracy 0.8319107025607354


In [1]:
loaded_model = ClassificationModel("bert", "./outputs_bert/", use_cuda=False)
result, model_outputs, wrong_preds = loaded_model.eval_model(valid_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(valid_df['labels'], predictions), 'accuracy', accuracy_score(valid_df['labels'], predictions))

NameError: name 'ClassificationModel' is not defined

In [10]:
result, model_outputs, wrong_preds = loaded_model.eval_model(train_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(train_df['labels'], predictions), 'accuracy', accuracy_score(train_df['labels'], predictions))

  0%|          | 0/6090 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/762 [00:00<?, ?it/s]

f1 score: 0.8673777420004024 accuracy 0.8917898193760263


In [4]:
loaded_model = ClassificationModel("bert", "./outputs_bert/", use_cuda=False)
writer = SummaryWriter("./")
writer.add_graph(loaded_model, valid_df)
writer.close()

ModuleNotFoundError: No module named 'caffe2'

In [13]:
from torchviz import make_dot
result, model_outputs, wrong_preds = loaded_model.eval_model(valid_df)
y = np.argmax(model_outputs, axis=-1)

In [17]:
make_dot(model_outputs.mean(), params=dict(loaded_model.model.named_parameters()))

TypeError: 'int' object is not callable

In [20]:
dict(loaded_model.model.named_parameters()).size()

AttributeError: 'dict' object has no attribute 'size'